### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Далее уже запускать код ниже

In [8]:
import numpy as np
import pandas as pd
import random

from urllib import request, parse

#metrics
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

import pickle
#import dill
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [9]:
TRAIN_DATASET_PATH = './project_data/train.csv'
TEST_DATASET_PATH = './project_data/test.csv'
#MACRO_INFO_PATH = './project_data/macro.csv'
#SAMPLE_SUBMISSION = './project_data/sample_submission.csv'
PREPARED_X_TRAIN_DATASET_PATH = './project_data/X_train.csv'
PREPARED_Y_TRAIN_DATASET_PATH = './project_data/y_train.csv'
PREPARED_X_VALID_DATASET_PATH = './project_data/X_valid.csv'
PREPARED_Y_VALID_DATASET_PATH = './project_data/y_valid.csv'

In [10]:
def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):
    """
    Функция выводит основные метрики для оценки модели: R2, MAE, MSE.
    
    """
    print("Train:" + "\n" + 
        "R2:\t" + str(round(r2(train_true_values, train_pred_values), 3)) + "\n" +
        "MAE:\t" + str(round(mae(test_true_values, test_pred_values), 3)) + "\n" +
        "MSE:\t" + str(round(mse(test_true_values, test_pred_values), 3)))
    print("Test:" + "\n" + 
        "R2:\t" + str(round(r2(test_true_values, test_pred_values), 3)) + "\n" +
        "MAE:\t" + str(round(mae(test_true_values, test_pred_values), 3)) + "\n" +
        "MSE:\t" + str(round(mse(test_true_values, test_pred_values), 3)))
    
    plt.figure(figsize=(12,8))
    
    plt.subplot(121)
    sns.scatterplot(x=train_pred_values, y=train_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Train sample prediction')
    
    plt.subplot(122)
    sns.scatterplot(x=test_pred_values, y=test_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Test sample prediction')

    plt.show()

In [11]:
X_train = pd.read_csv(PREPARED_X_TRAIN_DATASET_PATH)
y_train = pd.read_csv(PREPARED_Y_TRAIN_DATASET_PATH)

X_valid = pd.read_csv(PREPARED_X_VALID_DATASET_PATH)
y_valid = pd.read_csv(PREPARED_Y_VALID_DATASET_PATH)

In [12]:
X_valid.tail(3)

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,metro_km_walk,metro_km_avto,green_zone_km,park_km,water_treatment_km,kremlin_km,full_all,young_all,work_all,ekder_all,build_count_before_1920,build_count_1921-1945,state,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,raion_build_count_with_builddate_info,green_zone_part,indust_part,oil_chemistry_raion,big_market_raion
7338,53,53.0,1.0,NaN,NaN,NaN,NaN,NaN,5.700245,6.732362,0.183506,6.520123,13.997580,23.373697,7341,0.426100,1.906552,0.639559,0.000000,0.000000,NaN,1.000000,0.000000,0.000000,1.0,0.492840,0.075779,no,no
7339,32,17.0,6.0,12.0,1.0,1972.0,1.0,6.0,2.129673,1.746509,0.283741,2.465516,7.126955,16.470776,112221,0.158883,0.831475,0.289803,0.000000,0.004505,3.0,0.063063,0.846847,0.085586,222.0,0.153279,0.127376,no,no
7340,41,21.0,9.0,NaN,NaN,NaN,NaN,NaN,0.635096,0.635096,0.000000,3.881275,5.228332,15.891746,1318695,0.016942,0.106304,0.043290,0.008834,0.095406,NaN,0.358657,0.371025,0.166078,566.0,0.300284,0.033494,no,no


In [13]:
import urllib.request
import json      


def get_prediction(x):
    full_sq, num_room, build_year = x
    body = {'full_sq': full_sq, 
            'num_room': num_room,
            'build_year': build_year} 

    myurl = "http://0.0.0.0:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [15]:
get_prediction([25,1,1990])

URLError: <urlopen error [WinError 10049] The requested address is not valid in its context>

In [16]:
%%time
predictions = X_valid[['full_sq', 'num_room', 
                      'build_year']].apply(lambda x: get_prediction(x), 1)

URLError: <urlopen error [WinError 10049] The requested address is not valid in its context>